In [ ]:
import sys
!{sys.executable} -m pip install geopandas
!pip install folium

Load Fire Points Dataset 

In [ ]:

import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point

#load the historical fire dataset
#Read shapefile into a GeoDataFrame
fires_gdf = gpd.read_file("/Users/taimoorkiani/Downloads/NFDB_point/NFDB_point_20250519.shp")

#only take necessary cols from dataset
fires_gdf = fires_gdf[["LATITUDE", "LONGITUDE", "REP_DATE", "SIZE_HA"]]

#convert date to pandas datetime to filter/sort easily
fires_gdf["REP_DATE"] = pd.to_datetime(fires_gdf["REP_DATE"])

# Convert lat/lon coordinates into Point objects that GeoDataFrame can use for spatial queries
fires_gdf = gpd.GeoDataFrame(
    fires_gdf, 
    geometry=gpd.points_from_xy(fires_gdf["LONGITUDE"], fires_gdf["LATITUDE"]),
    crs="EPSG:4326"  #standard lat/lon (GPS coordinates)
)

#create a Canada lat/long grid
min_lat, max_lat = 41.0, 83.0
min_lon, max_lon = -141.0, -52.0

#step size(degrees) 0.5 ~ 50-60km, creates evenly spaced grid points
lat_steps = np.arange(min_lat, max_lat, 0.5)
lon_steps = np.arange(min_lon, max_lon, 0.5)

#create all possible grid points
grid_points = []
for lat in lat_steps:
  for lon in lon_steps:
    grid_points.append((lat, lon))

#convert list of tuples to DataFrame
grid_df = pd.DataFrame(grid_points, columns=["lat", "lon"])

#create a geometry column for grid points(new GeoDataFrame)
grid_gdf = gpd.GeoDataFrame(
    grid_df, #Original grid data

    #List comprehension creates Point object for each (lat, lon) pair
    # zip() pairs corresponding elements from two lists
    geometry=[
        Point(lon, lat)  #Point takes (x, y) = (lon, lat)
        for lat, lon in zip(grid_df['lat'], grid_df['lon'])
    ],
    crs="EPSG:4326" # Same coordinate system as fire data
)

# Reproject the grid to a projected CRS (meters)
grid_gdf = grid_gdf.to_crs(epsg=3347)

# Create 25km radius buffer zones around each grid point
# buffer() only works correctly in a projected CRS (meters)
grid_gdf['geometry'] = grid_gdf.geometry.buffer(25000)

# Reproject fire data to match the grid's CRS
fires_gdf = fires_gdf.to_crs(epsg=3347)

# sjoin finds all fire points that fall inside each grid cell's buffer
# predicate='contains' means: keep grid cells whose buffer contains a fire
joined = gpd.sjoin(grid_gdf, fires_gdf, predicate='contains')


# Grid cell gets 1 if its index appears in 'joined', else 0
# .isin() checks if each grid_gdf index exists in joined.index
grid_gdf["historical_fire"] = grid_gdf.index.isin(joined.index).astype(int)

#save as csv
#Drop geometry (CSV can't store shapely objects) before saving
grid_gdf.drop(columns="geometry").to_csv("canada_fire_grid.csv", index=False)

print("Canada wide grid saved as csv")



Canada wide grid saved as csv
